In [1]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

from secrets import GENIUS_API_TOKEN

In [2]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Lana Del Rey', 2)

Found 2 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-love-lyrics']

In [3]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

I've seen the world, done it all
Had my cake now
Diamonds, brilliant, and Bel Air now
Hot summer nights, mid-July
When you and I were forever wild
The crazy days, city lights
The way you'd play with me like a child
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
I've seen the world, lit it up as my stage now
Channeling angels in the new age now
Hot summer days, rock and roll
The way you'd play for me at your show
And all the ways I got to know
Your pretty face and electric soul
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
Dear Lord, when I get to heaven
Please, let me bring my man
When he comes, tell me that You'll let 

In [23]:
def write_lyrics_to_file(artist_name, song_count):
    urls = request_song_url(artist_name, song_count)
    count = 0
    for url in urls:
        f = open('lyrics/' + url[19:] + f'{count}' + '.txt', 'wb')
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
        f.close()
        print(f'Song {url[19:]} Written')
        count += 1
    print('Wrote {} files'.format(song_count))
  
# DEMO  
write_lyrics_to_file('Frank Ocean', 100)

Found 100 songs by Frank Ocean
Song Frank-ocean-thinkin-bout-you-lyrics Written
Song Frank-ocean-nights-lyrics Written
Song Frank-ocean-pink-matter-lyrics Written
Song Frank-ocean-chanel-lyrics Written
Song Frank-ocean-self-control-lyrics Written
Song Frank-ocean-pyramids-lyrics Written
Song Frank-ocean-ivy-lyrics Written
Song Frank-ocean-nikes-lyrics Written
Song Frank-ocean-super-rich-kids-lyrics Written
Song Frank-ocean-lost-lyrics Written
Song Frank-ocean-pink-white-lyrics Written
Song Frank-ocean-bad-religion-lyrics Written
Song Frank-ocean-novacane-lyrics Written
Song Frank-ocean-forrest-gump-lyrics Written
Song Frank-ocean-white-ferrari-lyrics Written
Song Frank-ocean-solo-lyrics Written
Song Frank-ocean-seigfried-lyrics Written
Song Frank-ocean-swim-good-lyrics Written
Song Frank-ocean-biking-lyrics Written
Song Frank-ocean-futura-free-lyrics Written
Song Frank-ocean-solo-reprise-lyrics Written
Song Frank-ocean-sweet-life-lyrics Written
Song Frank-ocean-godspeed-lyrics Written
